In [60]:
import pandas as pd
import numpy as np
from pathlib import Path

DATASET_PATH = 'datasets'
CLEANED_PATH = 'cleaned_datasets'

# Messages

In [2]:
messages = pd.read_csv(Path(DATASET_PATH, 'messages.csv'), 
                       parse_dates=[
                            'clicked_first_time_at', 
                            'clicked_last_time_at',
                            'opened_first_time_at',
                            'opened_last_time_at', 
                            'unsubscribed_at',
                            'hard_bounced_at',
                            'soft_bounced_at',
                            'complained_at',
                            'purchased_at',
                            'blocked_at',
                            'created_at',
                            'sent_at',
                            'updated_at'
                        ],
                        date_format='%Y-%m-%d %H:%M:%S.%f',
                        true_values=['t'], false_values=['f'],
                        dtype={
                            'message_id':'string',
                            'campaign_id':'int32',
                            'message_type':'category',
                            'channel':'category',
                            'platform':'category', 
                            'email_provider':'string', 
                            'user_device_id':'int16', 
                            'user_id':'int32'                           
                        }
                        ).drop(columns=['category','stream','id','date'])

In [59]:
messages['message_id'] = messages['message_id'].str.replace(
    r'(\w{8})(\w{4})(\w{4})(\w{3})-(\w)(\w{12})', 
    r'\1-\2-\3-\4\5-\6', regex=True)

In [8]:
unique_messages = messages[['campaign_id', 'message_type']].drop_duplicates().reset_index(drop=True)
unique_messages['id'] = unique_messages.index + 1
messages = messages.merge(unique_messages, on=['campaign_id', 'message_type'], how='left')

In [9]:
clients = messages[['client_id','user_id','user_device_id']].drop_duplicates()
users = messages['user_id'].drop_duplicates()

## Behavior

In [10]:
behaviors_cols = [col.replace('is_', '') 
                      for col in messages.columns 
                      if col.startswith('is_')]

message_behavior = [ 
    (
        messages[['message_id', f'is_{b}']].rename(columns={f'is_{b}': 'flag'})
        .assign(
            type = b,
            happened_first_time = messages.get(f'{b}_first_time_at', messages.get(f'{b}_at', pd.NaT)),
            happened_last_time = messages.get(f'{b}_last_time_at', pd.NaT)
        ))
    for b in behaviors_cols
]

message_behavior = pd.concat(message_behavior, ignore_index=True)

message_behavior = message_behavior[message_behavior['flag'] == True]\
    .drop(columns=['flag'])\
    .sort_values(['message_id','happened_first_time'])\
    .set_index(['message_id','type'])

message_behavior.to_csv(Path(CLEANED_PATH, 'message_behavior.csv'))
del(message_behavior)

OSError: Cannot save file into a non-existent directory: 'cleaned_datasets'

## Message_sent

In [7]:
message_sent = messages[['message_id','id','client_id',
                         'email_provider','platform',
                         'created_at','updated_at', 'sent_at']].set_index('message_id')
message_sent.to_csv(Path(CLEANED_PATH, 'message_sent.csv'))
del(message_sent)

In [9]:
messages = messages[['id','campaign_id','message_type','channel']].drop_duplicates(['campaign_id','message_type']).set_index('id')
messages.to_csv(Path(CLEANED_PATH,'messages.csv'))
del(messages)

# Campaigns

In [10]:
campaigns = pd.read_csv('datasets/campaigns.csv',
                        parse_dates=[
                            'started_at', 
                            'finished_at'
                        ],
                        dtype={
                            'campaign_type':'category',
                            'channel':'category',
                            'topic':'string',
                            'total_count':'Int32',
                            'ab_test':'boolean',
                            'warmup_mode':'boolean',
                            'hour_limit':'Int32',
                            'subject_length':'Int16',
                            'subject_with_personalization':'boolean',
                            'subject_with_deadline':'boolean',
                            'subject_with_emoji':'boolean',
                            'subject_with_bonuses':'boolean',
                            'subject_with_discount':'boolean',
                            'subject_with_saleout':'boolean',
                            'is_test':'boolean',
                            'position':'Int16' 
                        }
                        )\
                        .fillna({'ab_test':False,
                                 'warmup_mode':False,
                                 'is_test':False})\
                        
campaigns.index.names = ['campaign_pk']
campaigns.shape

(1907, 19)

In [11]:
campaigns = campaigns[~(
    (campaigns['is_test'] == True)
    ### Bulk rules
    | (campaigns['campaign_type'] == 'bulk') 
        & (campaigns['started_at'].isna() 
           | (campaigns['warmup_mode'] == True) & campaigns['hour_limit'].isna())
    ### Trigger rules
    | (campaigns['campaign_type'] == 'trigger')
        & campaigns['position'].isna() 
    )].drop(columns=['is_test'])
campaigns.shape

(1892, 18)

In [12]:
# (From description): Multichannel is a kind of transactional campaign when a retailer 
# tries to deliver important information from the cheapest channel 
# to the more expensive, depending on which channels recipient uses.
campaigns[campaigns['channel']=='multichannel'].head(3)
# But these multichannel campaign subset related only to trigger campaign_type

,id,campaign_type,channel,topic,started_at,finished_at,total_count,ab_test,warmup_mode,hour_limit,subject_length,subject_with_personalization,subject_with_deadline,subject_with_emoji,subject_with_bonuses,subject_with_discount,subject_with_saleout,position
campaign_pk,,,,,,,,,,,,,,,,,,
1830,20519,trigger,multichannel,abandoned cart,NaT,NaT,<NA>,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0
1836,20896,trigger,multichannel,abandoned cart,NaT,NaT,<NA>,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4
1837,21379,trigger,multichannel,abandoned cart,NaT,NaT,<NA>,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4


In [13]:
# (From description): SMS doesn’t have subject
campaigns[campaigns['subject_length'].notna()]['channel'].value_counts(dropna=False)
# But actually sms has subject
# I'll drop subject for sms because it doesn't meet business requirements

channel
mobile_push     1394
email            479
sms                1
multichannel       0
Name: count, dtype: int64

## Types of compaigns

In [14]:
bulk_cols = [
        # Only bulk campaigns have start date and finish date
        'started_at', 
        'finished_at', # Finish date is the date/time when campaign did sent the *final* message -> can be NaN
        'total_count', # Says how many recipients was in bulk campaign when it was sent.
        'warmup_mode', #  Campaign works days or weeks
        'hour_limit', # with rule: how many messages can be sent every hour
        'ab_test' #  A/B test mode (campaign to a limited audience)
    ]
bulks = campaigns[campaigns['campaign_type']=='bulk'][bulk_cols]
bulks.to_csv(Path(CLEANED_PATH, 'bulks.csv'))
del(bulks)

subject_cols = [
        'subject_length',
        'subject_with_personalization',
        'subject_with_deadline',
        'subject_with_emoji',
        'subject_with_bonuses',
        'subject_with_discount',
        'subject_with_saleout'
    ]
campaign_subject = campaigns[~campaigns['channel']\
                             .isin(['sms', 'multichannel'])][subject_cols]
campaign_subject.to_csv(Path(CLEANED_PATH, 'campaign_subject.csv'))
del(campaign_subject)

trigger_cols = ['position']
triggers = campaigns[campaigns['campaign_type']=='trigger'][trigger_cols]
triggers.to_csv(Path(CLEANED_PATH, 'triggers.csv'))
del(triggers)

campaigns = campaigns.drop(columns=bulk_cols 
                                + subject_cols 
                                + trigger_cols)
campaigns.to_csv(Path(CLEANED_PATH, 'campaigns.csv'))
del(campaigns)

# Events

In [72]:
events = pd.read_csv('datasets/events.csv',
                     parse_dates=['event_time'],
                     date_format='%Y-%m-%d %H:%M:%S UTC',
                     dtype={
                         'event_type':'category',
                         'product_id':'int32',
                         'category_id':'int64',
                         'category_code':'category',
                         'brand':'category',
                         'price':'float32',
                         'user_id':'int32',
                         'user_session':'string'
                     }
                     )

In [16]:
users = pd.concat([users, 
                   events['user_id'].drop_duplicates()])

In [73]:
# Function to choose the representative value from category_code within each group.
def choose_representative(series):
    non_null = series.dropna()
    if non_null.empty: return np.nan
    else:
        mode_vals = non_null.mode()
        if not mode_vals.empty:
            return mode_vals.iloc[0]
        else:
            return non_null.iloc[0]

In [74]:
unique_products = events.groupby(['product_id', 'category_id'], as_index=False)['category_code']\
    .agg(choose_representative)
unique_products['product_pk'] = unique_products.index + 1
events = events.drop(columns='category_code').merge(unique_products, on=['product_id', 'category_id'], how='left')

In [75]:
unique_produc_cards = events[['product_pk', 'brand']].drop_duplicates().reset_index(drop=True)
unique_produc_cards['product_card_pk'] = unique_produc_cards.index + 1
events = events.merge(unique_produc_cards, on=['product_pk', 'brand'], how='left')

In [76]:
events

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,category_code,product_pk,product_card_pk
0,2019-10-01 02:28:19,view,1004434,2053013555631882655,samsung,257.149994,548844080,dcc9b83f-0332-4f04-8f3f-51ea311a5443,electronics.smartphone,789,1
1,2019-10-01 02:39:53,view,1004449,2053013555631882655,xiaomi,205.669998,549110945,fbb8ecf9-3d1e-49d4-80bb-aec6ed2f0033,electronics.smartphone,806,2
2,2019-10-01 02:49:08,view,1306798,2053013558920217191,acer,332.049988,555119848,7d8505d4-4756-4d60-8aa2-3938b752d8bc,computers.notebook,2670,3
3,2019-10-01 02:49:41,view,1306798,2053013558920217191,acer,332.049988,555119848,7d8505d4-4756-4d60-8aa2-3938b752d8bc,computers.notebook,2670,3
4,2019-10-01 02:59:27,view,5800778,2053013553945772349,kicx,15.440000,549040537,bc69321e-4aa0-45fc-b98e-9055032153fc,electronics.audio.subwoofer,20794,4
...,...,...,...,...,...,...,...,...,...,...,...
1280399,2019-12-31 21:03:40,view,4800107,2232732079706079299,kingston,102.680000,554776088,4f037e9b-6b55-47ee-86c2-7431802d801c,sport.bicycle,15780,47822
1280400,2019-12-31 21:10:48,view,12708575,2053013554566529381,matador,35.750000,550313875,2f11fec0-b08a-4c49-abaa-16e235ed30ca,computers.desktop,41207,45343
1280401,2019-12-31 21:46:43,view,10300401,2232732104888681081,silverlit,9.020000,560249531,e4a8dd0d-1790-45e4-b9a2-90da2691bca4,apparel.scarf,32107,29514
1280402,2019-12-31 22:50:13,view,1004781,2232732093077520756,huawei,230.380005,554930302,9dd09620-93e4-4781-a477-22b6809f5d72,construction.tools.light,1081,22901


In [19]:
products = events[['product_pk','product_id', 'category_id', 'category_code']].drop_duplicates().set_index('product_pk')
products.to_csv(Path(CLEANED_PATH, 'products.csv'))
del(products)
product_cards = events[['product_card_pk','product_pk','brand']].drop_duplicates().set_index('product_card_pk')
product_cards.to_csv(Path(CLEANED_PATH, 'product_cards.csv'))
del(product_cards)

In [20]:
events = events.drop_duplicates(['product_card_pk','user_id','event_time'])\
                [['product_card_pk','user_id','event_time',
                 'event_type','user_session','price']]
events.to_csv(Path(CLEANED_PATH, 'events.csv'))
del(events)

# Client first purchase

In [21]:
first_purchase = pd.read_csv('datasets/client_first_purchase_date.csv',
                             parse_dates=['first_purchase_date'],
                             date_format='%Y-%m-%d',
                             dtype={'user_id':'int32',
                             'user_device_id':'int16'})

In [22]:
clients = pd.concat([clients,
                     first_purchase[['client_id',
                                     'user_id',
                                     'user_device_id']].drop_duplicates()])
clients.to_csv(Path(CLEANED_PATH, 'clients.csv'))
del(clients)
users = pd.concat([users, 
                   first_purchase['user_id'].drop_duplicates()])
users.to_csv(Path(CLEANED_PATH, 'users.csv'))
del(users)

In [23]:
first_purchase = first_purchase[['client_id',
                                 'first_purchase_date']].drop_duplicates()
first_purchase.to_csv(Path(CLEANED_PATH, 'first_purchase.csv'))
del(first_purchase)

# Friends

In [24]:
friends = pd.read_csv('datasets/friends.csv',
                        dtype={
                            'friend1':'int32',
                            'friend2':'int32'
                        })
friends = pd.DataFrame(np.sort(friends.values, axis=1), columns=friends.columns)
friends.to_csv(Path(CLEANED_PATH, 'friends.csv'))
del(friends)